In [1]:
# Importing required module
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings('ignore');
import os

os.chdir('C:\\Users\\Ravi\\Downloads\\OneDrive_1_9-14-2021')

In [2]:
# Reading Data
request = pd.read_csv("T_REQUEST.csv",sep=",")

version = pd.read_csv("T_REQUEST_VERSION.csv",sep=",")
version_col = ['requester_id', 'addressee_id', 'direction','request_id', 'version', 'status',
               'pickup_charge', 'damage_protection_insurance','user_has_visited', 'supplier_has_visited']
version = version[version_col]

tracking = pd.read_csv("T_USER_TRACKING.csv",sep=",")

In [3]:
# list of success status
success_status = ['PICKED_UP', 'ACCEPTED','READY_FOR_PICKUP', 'EMPTY_RETURNED'] 

# creating dataframe of request id when last version is success. 
last_version = version.groupby(['request_id','requester_id','addressee_id'], as_index=False)['status'].last()
success_data = last_version[last_version.status.isin(success_status)]
success_data.head(10)

,request_id,requester_id,addressee_id,status
3380,3387,52,4,EMPTY_RETURNED
3644,3651,36,57,EMPTY_RETURNED
3916,3923,81,39,EMPTY_RETURNED
3939,3946,22,36,EMPTY_RETURNED
3956,3963,22,36,EMPTY_RETURNED
4055,4062,22,13,EMPTY_RETURNED
4096,4103,22,13,EMPTY_RETURNED
4148,4155,81,23,EMPTY_RETURNED
4159,4166,36,19,EMPTY_RETURNED
4160,4167,22,13,EMPTY_RETURNED


In [4]:
# Number of successfull orders
total_success = success_data.groupby(['addressee_id']).agg( total_success=('request_id','count'))
total_success

,total_success
addressee_id,
1,23
2,1085
3,1
4,112
5,47
...,...
2303,2
2337,2
2370,3


In [5]:
success_requests = success_data.merge(request[['id','created_date']],left_on='request_id', right_on='id', how='left')
success_requests.drop(['id','requester_id'], axis=1, inplace=True)

pd.merge(success_requests,tracking,left_on=['addressee_id','created_date'], right_on=['company_id','created_date'], how='inner')

,request_id,addressee_id,status,created_date,id,event,additional_info,user_id,impersonated,company_id
0,21685,62,EMPTY_RETURNED,2018-06-06 11:15,1252,Get Requests For Request Overview,NaN,360,1,62
1,21685,62,EMPTY_RETURNED,2018-06-06 11:15,1253,Request Overview,NaN,360,1,62
2,21685,62,EMPTY_RETURNED,2018-06-06 11:15,1254,Request Overview,NaN,360,1,62
3,21685,62,EMPTY_RETURNED,2018-06-06 11:15,1255,Get Requests For Request Overview,NaN,360,1,62
4,21685,62,EMPTY_RETURNED,2018-06-06 11:15,1257,View Request Details,Request Id: 21685,360,1,62
...,...,...,...,...,...,...,...,...,...,...
805,128623,155,READY_FOR_PICKUP,2021-04-22 06:20,1657643,Get All Requests,NaN,1949,0,155
806,128623,155,READY_FOR_PICKUP,2021-04-22 06:20,1657644,View Request Details,Request Id: 128623,1949,0,155
807,128623,155,READY_FOR_PICKUP,2021-04-22 06:20,1657646,Accept Request,Request Id: 128623,1949,0,155
808,128623,155,READY_FOR_PICKUP,2021-04-22 06:20,1657647,View Request Details,Request Id: 128623,1949,0,155


In [6]:
# making list of successful company ids
success_company = success_data['addressee_id'].unique().tolist()

# creating dataframe of tracking for succful companies.
success_tracking = tracking[tracking.company_id.isin(success_company)]
success_tracking.head()

,id,event,additional_info,user_id,impersonated,created_date,company_id
0,1,Get Requests For Dashboard,NaN,413,1,2018-06-05 16:08,81
1,2,Get Requests For Dashboard,NaN,507,0,2018-06-05 16:08,115
2,3,Get Requests For Dashboard,NaN,507,0,2018-06-05 16:12,115
3,4,Get Requests For Dashboard,NaN,495,0,2018-06-05 16:12,6
4,5,Get Requests For Dashboard,NaN,3,0,2018-06-05 16:12,36


In [7]:
events = success_tracking.groupby(['company_id','event'], as_index=False)['id'].count()
events.sort_values(['company_id','id'],inplace = True)
events.head()

,company_id,event,id
4,1,Send Request,2
5,1,View Balance Overview,4
3,1,Reject Request,22
0,1,Get All Requests,25
7,1,View Tracking Page,27


In [8]:
request.head()

,id,created_by,created_date,last_modified_by,last_modified_date,carrier_on_turn_id,emails_deactivated,manual_request,requesters_agent_msg,addressees_agent_msg,...,assisted_transaction,insurance_payer,active_matchmaking,insurance_only,critical,leasing,prepayment_of_charge,no_payment_guarantee,grey_zone_agreed,has_damage_claim
0,1,ulrich.bartels@hamburgsud.com,2015-06-30 11:19,system,2016-01-25 23:00,NaN,0,0,NaN,NaN,...,0,NaN,1,0,0,0,0,0,0,0
1,2,malini.k@uasc.net,2015-07-06 08:56,system,2016-01-25 23:00,NaN,0,0,NaN,NaN,...,0,NaN,1,0,0,0,0,0,0,0
2,3,tpartridge@triu.com,2015-08-05 16:51,system,2016-01-25 23:00,NaN,0,0,NaN,NaN,...,0,NaN,1,0,0,0,0,0,0,0
3,4,tpartridge@triu.com,2015-08-11 04:28,tpartridge@triu.com,2015-08-11 04:28,NaN,0,0,NaN,NaN,...,0,NaN,1,0,0,0,0,0,0,0
4,5,tpartridge@triu.com,2015-08-12 00:57,system,2016-01-25 23:00,NaN,0,0,NaN,NaN,...,0,NaN,1,0,0,0,0,0,0,0
